In [1]:
!pip install transformers pillow

In [2]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

In [6]:
# # Load BLIP-2 Model and Processor
# model_name = "Salesforce/blip2-flan-t5-xl"
# processor = Blip2Processor.from_pretrained(model_name)
# model = Blip2ForConditionalGeneration.from_pretrained(model_name)


**Simple VQA using llama vision using groq api , sarvam api for tts and stt**

In [7]:
!pip install requests


In [8]:
!pip install requests pillow pyttsx3


In [10]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install pydub

In [48]:
import requests
import base64
from google.colab import files
from PIL import Image

# API details for Sarvam Speech-to-Text
SARVAM_API_URL = "https://api.sarvam.ai/speech-to-text"
API_SUBSCRIPTION_KEY = "9ad8976b-6ad1-4994-bea4-87c914bda6db"  # Replace with your Sarvam API key
LANGUAGE_CODE = "hi-IN"  # Language code (e.g., hi-IN for Hindi)
MODEL = "saarika:v1"  # Model version

# Groq API details for Vision Model
from groq import Groq
groq_api_key ="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"
groq_client = Groq(
    api_key=groq_api_key,
)

# Function to transcribe audio with Sarvam
def transcribe_audio(audio_path):
    # Ensure the file has the correct extension
    if not audio_path.lower().endswith((".wav", ".mp3")):
        print("Error: The file must be in .wav or .mp3 format.")
        return None

    # Open and send the file to the API
    with open(audio_path, "rb") as file:
        files = {"file": (audio_path, file, "audio/wav")}  # Explicitly set the file MIME type
        data = {
            "language_code": LANGUAGE_CODE,
            "model": MODEL,
            "with_timestamps": "true"
        }
        headers = {"api-subscription-key": API_SUBSCRIPTION_KEY}

        print("Sending audio to Sarvam API...")
        response = requests.post(SARVAM_API_URL, headers=headers, files=files, data=data)

        if response.status_code == 200:
            result = response.json()
            transcript = result.get("transcript", "")
            print("Transcription successful:", transcript)
            return transcript
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None
def transcribe_and_translate_audio(audio_path):
    # Ensure the file has the correct extension
    if not audio_path.lower().endswith((".wav", ".mp3")):
        print("Error: The file must be in .wav or .mp3 format.")
        return None

    # Open and send the file to the Speech-to-Text-Translate API
    with open(audio_path, "rb") as file:
        files = {"file": (audio_path, file, "audio/wav")}  # Explicitly set the file MIME type
        data = {
            "model": "saaras:v1",
            "prompt": ""  # Optional: Add a context-specific prompt here
        }
        headers = {"api-subscription-key": API_SUBSCRIPTION_KEY}

        print("Sending audio to Sarvam Speech-to-Text-Translate API...")
        response = requests.post("https://api.sarvam.ai/speech-to-text-translate", headers=headers, files=files, data=data)

        if response.status_code == 200:
            result = response.json()
            transcript = result.get("transcript", "")
            language_code = result.get("language_code", "unknown")
            print(f"Transcription successful (Detected Language: {language_code}): {transcript}")
            return transcript
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None
from PIL import Image
import base64
import io

def encode_image_to_base64(image_path):
    """Convert image to base64 string"""
    with Image.open(image_path) as img:
        # Convert image to RGB if it's not
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # Create a byte buffer
        buffer = io.BytesIO()
        img.save(buffer, format='JPEG')
        image_bytes = buffer.getvalue()

        # Encode to base64
        base64_image = base64.b64encode(image_bytes).decode('utf-8')
        return f"data:image/jpeg;base64,{base64_image}"

def process_image_and_query(image_path, query):
    # Convert image to base64
    image_data_url = encode_image_to_base64(image_path)

    # Prepare the messages for LLaVA model
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": query
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_data_url
                    }
                }
            ]
        }
    ]

    # Send request to Groq's LLaVA model
    print("Sending query to LLaVA model...")
    completion = groq_client.chat.completions.create(
        model="llava-v1.5-7b-4096-preview",
        messages=messages,
        temperature=0.7,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None
    )

    return completion.choices[0].message.content


import requests
from google.colab import files

def translate_text(text, target_lang_code):
    """
    Translates text to target language using Sarvam.ai API
    Args:
        text (str): Text to translate
        target_lang_code (str): Target language code (e.g., 'hi-IN', 'ta-IN', 'te-IN', etc.)
    Returns:
        str: Translated text
    """
    try:
        url = "https://api.sarvam.ai/translate"

        payload = {
            "input": text,
            "source_language_code": "en-IN",  # Source is English
            "target_language_code": target_lang_code,
            "speaker_gender": "Male",
            "mode": "formal",
            "model": "mayura:v1",
            "enable_preprocessing": True
        }

        headers = {
            "Content-Type": "application/json",
            "api-subscription-key":  API_SUBSCRIPTION_KEY # Replace with your Sarvam.ai API key
        }

        response = requests.post(url, json=payload, headers=headers)

        if response.status_code == 200:
            return response.json()['translated_text']  # Adjust based on actual API response structure
        else:
            print(f"Translation failed with status code: {response.status_code}")
            print(f"Error message: {response.text}")
            return None

    except Exception as e:
        print(f"Translation error: {e}")
        return None

def get_supported_languages():
    """
    Returns a dictionary of supported languages and their codes
    """
    return {
        "Hindi": "hi-IN",
        "Tamil": "ta-IN",
        "Telugu": "te-IN",
        "Kannada": "kn-IN",
        "Malayalam": "ml-IN",
        "Marathi": "mr-IN",
        "Bengali": "bn-IN",
        "Gujarati": "gu-IN",
        "Punjabi": "pa-IN",
        "Odia": "or-IN",
        # Add more languages as supported by Sarvam.ai
    }


def text_to_speech(text, target_language_code="hi-IN"):
    """
    Converts text to speech using the provided API and saves the audio as a .wav file.
    """
    url = "https://api.sarvam.ai/text-to-speech"

    payload = {
        "inputs": [text],
        "target_language_code": target_language_code,
        "speaker": "meera",  # You can change the speaker here
        "pitch": 0,
        "pace": 1.65,
        "loudness": 1.5,
        "speech_sample_rate": 8000,
        "enable_preprocessing": True,
        "model": "bulbul:v1"
    }
    headers = {
        "Content-Type": "application/json",
        "api-subscription-key": API_SUBSCRIPTION_KEY
    }

    # Sending request to the Sarvam AI API
    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        # Get the audio content (assuming it's in MP3 format)
        audio_data = response.json()['audios'][0]  # Extracting the base64 audio data
        return audio_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Function to convert base64 audio data to WAV and play it
def save_and_play_audio(base64_audio_data, output_path='output_audio.wav'):
    """
    Decodes base64 audio data, saves it as a .wav file, and plays it.
    """
    # Decode the base64 audio data
    audio_data = base64.b64decode(base64_audio_data)

    # Save the audio data as a WAV file
    with open(output_path, 'wb') as audio_file:
        audio_file.write(audio_data)

    # Play the audio
    ipd.Audio(output_path)

def chatbot_colab_translate():
    print("Welcome to the Visual Assistant Chatbot in Colab!")

    # Display supported languages
    supported_langs = get_supported_languages()
    print("\nSupported Languages:")
    for lang, code in supported_langs.items():
        print(f"- {lang} ({code})")

    # Get target language from user
    print("\nEnter the language code for translation (e.g., 'hi-IN' for Hindi):")
    target_language = input().strip()

    if target_language not in supported_langs.values():
        print(f"Warning: '{target_language}' might not be supported. Proceeding anyway...")

    # Step 1: Upload audio file for transcription
    print("\nPlease upload an audio file (.wav or .mp3) for transcription and translation.")
    audio_upload = files.upload()  # User uploads an audio file
    audio_path = list(audio_upload.keys())[0]  # Get the uploaded file path

    # Transcribe the uploaded audio
    transcript = transcribe_and_translate_audio(audio_path)
    if not transcript:
        print("Audio transcription failed. Please ensure the file is in .wav or .mp3 format and try again.")
        return

    print(f"Transcript: {transcript}")

    # Step 2: Upload an image for visual question answering
    print("\nPlease upload an image file for analysis.")
    image_upload = files.upload()  # User uploads an image file
    image_path = list(image_upload.keys())[0]  # Get the uploaded file path

    # Step 3: Use the translated transcription as the query for the vision model
    vision_response = process_image_and_query(image_path, transcript)

    # Step 4: Translate the final vision response to target language
    translated_response = translate_text(vision_response, target_language)

    print(f"\nChatbot Response (English): {vision_response}")
    print(f"Chatbot Response (Translated): {translated_response}")

    # Step 5: Convert the translated response to speech and play it
    base64_audio_data = text_to_speech(translated_response,target_language)

    if base64_audio_data:
        save_and_play_audio(base64_audio_data)

def main():
    """
    Main function with error handling
    """
    try:
        import requests
    except ImportError:
        !pip install requests

    try:
        chatbot_colab_translate()
    except KeyboardInterrupt:
        print("\nProcess interrupted by user.")
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        print("Please make sure you have set up the API key correctly and have internet connection.")

if __name__ == "__main__":
    main()


Welcome to the Visual Assistant Chatbot in Colab!

Supported Languages:
- Hindi (hi-IN)
- Tamil (ta-IN)
- Telugu (te-IN)
- Kannada (kn-IN)
- Malayalam (ml-IN)
- Marathi (mr-IN)
- Bengali (bn-IN)
- Gujarati (gu-IN)
- Punjabi (pa-IN)
- Odia (or-IN)

Enter the language code for translation (e.g., 'hi-IN' for Hindi):
hi-IN

Please upload an audio file (.wav or .mp3) for transcription and translation.


Saving hindi_question.wav to hindi_question (14).wav
Sending audio to Sarvam Speech-to-Text-Translate API...
Transcription successful (Detected Language: hi-IN): What all things are there in this picture?
Transcript: What all things are there in this picture?

Please upload an image file for analysis.


Saving heartwarming-moment-between-dog-cat-600nw-2432338827 (1).webp to heartwarming-moment-between-dog-cat-600nw-2432338827 (1) (13).webp
Sending query to LLaVA model...

Chatbot Response (English): In this picture, there is a dog and a cat playing together in a grassy area. They are engaging in a playful interaction, most likely running or rolling around on the grass. This image captures the amusing scene of a dog and a cat enjoying each other's company and sharing a fun moment.
Chatbot Response (Translated): इस तस्वीर में एक कुत्ता और एक बिल्ली एक हरे मैदान में एक साथ मस्ती करते हुए दिखाई दे रहे हैं। ऐसा लगता है कि वे या तो घास पर दौड़ रहे हैं या घूम रहे हैं। यह तस्वीर एक हल्के-फुल्के पल को दर्शाती है जहाँ एक कुत्ता और एक बिल्ली एक साथ अच्छा समय बिता रहे हैं।
